<a href="https://colab.research.google.com/github/deskskeleton/YouTube-GenderGap-Investigation/blob/master/get_youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copy of the .py file thales provided us with. remember to change api keys for yours and to partition via id's


In [ ]:
import os
import pandas as pd
from re import sub
import googleapiclient.discovery
from uritemplate import api

In [ ]:
def init_service():
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    api_service_name = "youtube"
    api_version = "v3"
    API_KEY = "----"  # open(".API_KEY").read().strip(), USE YOUR OWN KEY HERE
    return googleapiclient.discovery.build(
        api_service_name, api_version, developerKey=API_KEY
    )

In [ ]:
#creating list of the quarter of channel id's
  id_list = [UCkvK_5omS-42Ovgah8KRKtg,UCbO9bltbkYwa56nZFQx6XJg, UCbO9bltbkYwa56nZFQx6XJg, UCCvoAe__WFYMNAEN-C-CtYA, UCucot-Zp428OwkyRm2I7v2Q, UCDHQbU57NZilrhbuZNbQcRA, UC3EFKdXAU99j3ppGgvTz7XQ, UCFn4TEi42U-WHYjiqaxpp3w]

In [ ]:
def get_channel_info(api_service, channel_id):
    return (
        api_service.channels()
        .list(id=channel_id, part="contentDetails,snippet,statistics")
        .execute()
    )


In [ ]:
def get_channel_videos(api_service, playlist_id, max_videos=10):
    videos = []
    next_page_token = None
    n_videos = 0
    while True:
        response = (
            api_service.playlistItems()
            .list(
                playlistId=playlist_id,
                part="contentDetails,snippet",
                maxResults=10,
                pageToken=next_page_token,
            )
            .execute()
        )
        videos.extend(response["items"])
        next_page_token = response.get("nextPageToken")
        n_videos += 50
        if next_page_token is None or n_videos >= max_videos:
            break
    return videos

In [ ]:
def get_video_stats(api_service, video_id):
    return api_service.videos().list(id=video_id, part="statistics").execute()


In [ ]:
def get_video_comments(api_service, video_id, max_comments=10000):
    comments = []
    next_page_token = None
    n_comments = 0
    while True:
        response = (
            api_service.commentThreads()
            .list(
                videoId=video_id,
                part="snippet",
                maxResults=100,
                textFormat="plainText",
                pageToken=next_page_token,
            )
            .execute()
        )
        comments.extend(response["items"])
        next_page_token = response.get("nextPageToken")
        n_comments += 100
        if next_page_token is None or n_comments >= max_comments:
            break
    return comments

In [ ]:
# channel_id1,channel_id2,...
def retrieve_full_channel_data(api_service, channel_ids):
    channels_dataset = []
    videos_dataset = []
    comments_dataset = []
    # Retrieving the channels
    channel_info = get_channel_info(api_service, channel_ids)
    for channel in channel_info["items"]:
        channel_id = channel["id"]
        print(channel_id)
        # Filtering the response to get the relevant info
        channels_dataset.append(filter_channel_info(channel))
        # Retrieving the videos. We need the playlist id first!
        playlist_id = channel["contentDetails"]["relatedPlaylists"]["uploads"]
        videos = get_channel_videos(api_service, playlist_id)
        # Retrieving video stats + comments
        for video in videos:
            video_id = video["contentDetails"]["videoId"]
            print(f"video_id: {video_id}")
            video_stats = get_video_stats(api_service, video_id)
            videos_dataset.append(filter_video_info(video, video_stats["items"][0]))
            if (
                "commentCount" in video_stats["items"][0].get("statistics")
                and int(video_stats["items"][0].get("statistics").get("commentCount"))
                > 0
            ):
                comments = get_video_comments(api_service, video_id)
                for comment in comments:
                    filtered_comment_info = filter_comment_info(comment)
                    filtered_comment_info.extend([video_id, channel_id])
                    comments_dataset.append(filtered_comment_info)
            else:
                comments_dataset.append(["comments disabled", video_id, channel_id])
    return (channels_dataset, videos_dataset, comments_dataset)


In [ ]:
#
def filter_channel_info(response_item):
    id = response_item.get("id")
    title = response_item.get("snippet").get("title")
    description = response_item.get("snippet").get("description")
    published_at = response_item.get("snippet").get("publishedAt")
    view_count = response_item.get("statistics").get("viewCount")
    sub_count = response_item.get("statistics").get("subscriberCount")
    video_count = response_item.get("statistics").get("videoCount")
    return [id, title, description, published_at, view_count, sub_count, video_count]


In [ ]:
#
def filter_video_info(video_response_item, video_stats_response_item):
    id = video_response_item.get("id")
    video_id = video_response_item.get("contentDetails").get("videoId")
    title = video_response_item.get("snippet").get("title")
    description = video_response_item.get("snippet").get("description")
    channel = video_response_item.get("snippet").get("channelTitle")
    view_count = video_stats_response_item.get("statistics").get("viewCount")
    like_count = video_stats_response_item.get("statistics").get("likeCount")
    dislike_count = video_stats_response_item.get("statistics").get("dislikeCount")
    favourite_count = video_stats_response_item.get("statistics").get("favoriteCount")
    comment_count = video_stats_response_item.get("statistics").get("commentCount")
    return [
        id,
        video_id,
        title,
        description,
        channel,
        view_count,
        like_count,
        dislike_count,
        favourite_count,
        comment_count,
    ]



In [ ]:
#
def filter_comment_info(comment_response_item):
    id = comment_response_item.get("id")
    text = (
        comment_response_item.get("snippet")
        .get("topLevelComment")
        .get("snippet")
        .get("textOriginal")
    )
    author_name = (
        comment_response_item.get("snippet")
        .get("topLevelComment")
        .get("snippet")
        .get("authorDisplayName")
    )
    author_id = (
        (
            comment_response_item.get("snippet")
            .get("topLevelComment")
            .get("snippet")
            .get("authorChannelId")
            .get("value")
        )
        if "authorChannelId"
        in comment_response_item.get("snippet").get("topLevelComment").get("snippet")
        else ""
    )
    like_count = (
        comment_response_item.get("snippet")
        .get("topLevelComment")
        .get("snippet")
        .get("likeCount")
    )
    published_at = (
        comment_response_item.get("snippet")
        .get("topLevelComment")
        .get("snippet")
        .get("publishedAt")
    )
    updated_at = (
        comment_response_item.get("snippet")
        .get("topLevelComment")
        .get("snippet")
        .get("updatedAt")
    )
    reply_count = comment_response_item.get("snippet").get("totalReplyCount")
    return [
        id,
        text,
        author_name,
        author_id,
        like_count,
        published_at,
        updated_at,
        reply_count,
    ]



In [ ]:
#
def create_channels_df(channels_dataset):
    return pd.DataFrame(
        channels_dataset,
        columns=[
            "id",
            "title",
            "description",
            "published_at",
            "view_count",
            "sub_count",
            "video_count",
        ],
    )


In [ ]:
#
def create_videos_df(videos_dataset):
    return pd.DataFrame(
        videos_dataset,
        columns=[
            "id",
            "video_id",
            "title",
            "description",
            "channel",
            "view_count",
            "like_count",
            "dislike_count",
            "favourite_count",
            "comment_count",
        ],
    )

